In [1]:
import pandas as pd

# EDA

Let' open the file and see what information there is inside

In [50]:
df = pd.read_excel('get_around_delay_analysis.xlsx')

df.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


In [78]:
df.shape

(21310, 7)

In [105]:
df.describe()

,rental_id,car_id,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,16346.000000,1841.000000,1841.000000
mean,549712.880338,350030.603426,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,67.000000,560823.000000,540.000000
max,576401.000000,417675.000000,71084.000000,575053.000000,720.000000


There are 21k reservation in the dataframe. Let's have a quick look at the data inside

In [73]:
import plotly.express as px

In [11]:
for column in df.columns:
    fig = px.histogram(df,x=column)
    fig.show()

After a first approach of the dataset, let's see if there are NaN in the dataframe

In [20]:
for column in df.columns : 
    print(column,':',df[column].nunique())

rental_id : 21310
car_id : 8143
checkin_type : 2
state : 2
delay_at_checkout_in_minutes : 1745
previous_ended_rental_id : 1788
time_delta_with_previous_rental_in_minutes : 25


In [55]:
df.isna().sum()/df.shape[0]*100

rental_id                                      0.000000
car_id                                         0.000000
checkin_type                                   0.000000
state                                          0.000000
delay_at_checkout_in_minutes                  23.294228
previous_ended_rental_id                      91.360863
time_delta_with_previous_rental_in_minutes    91.360863
dtype: float64

There are only 9% of the rental for which we have information regarding the previous rental. Here there is an important question of the meaning of the NaN. 
- If the data are really missing, it means there are some traceability issue in the database and the process
- If it means that the previous rental was more than 12h before, it means that there is not a high turnover on the car

Let's assume than NaN is meaning than the previous rental was more than 12hour before

In [51]:
import numpy as np

df['time_delta_with_previous_rental_in_minutes'] = df['time_delta_with_previous_rental_in_minutes'].fillna(721)


# Define the bins and labels
bins = [-np.inf, 30, 60, 180, 720, np.inf]
labels = ['<30 minutes', '30 minutes to 1h', '1h to 3h', '3h to 12h', '>12h']

# Categorize the time_delta_with_previous_rental_in_minutes column
df['time_vs_previous_rental_category'] = pd.cut(
    df['time_delta_with_previous_rental_in_minutes'],
    bins=bins,
    labels=labels,
    right=False  # Right bound excluded, consistent with "<30"
)

df.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,time_vs_previous_rental_category
0,505000,363965,mobile,canceled,NaN,NaN,721.0,>12h
1,507750,269550,mobile,ended,-81.0,NaN,721.0,>12h
2,508131,359049,connect,ended,70.0,NaN,721.0,>12h
3,508865,299063,connect,canceled,NaN,NaN,721.0,>12h
4,511440,313932,mobile,ended,NaN,NaN,721.0,>12h


In [52]:
# Step 1: Group by and count rental_id
grouped = df.groupby(['time_vs_previous_rental_category', 'state'], observed=False)['rental_id'].count().reset_index()
grouped.rename(columns={'rental_id': 'count'}, inplace=True)

# Step 2: Group by the same columns and calculate the sum
sum_grouped = df.groupby(['time_vs_previous_rental_category'], observed=False)['rental_id'].count().reset_index()
sum_grouped.rename(columns={'rental_id': 'sum'}, inplace=True)

result = pd.merge(grouped, sum_grouped, on=['time_vs_previous_rental_category', 'time_vs_previous_rental_category'])

result['percentage'] = result['count']/result['sum']*100
result


,time_vs_previous_rental_category,state,count,sum,percentage
0,<30 minutes,canceled,35,279,12.544803
1,<30 minutes,ended,244,279,87.455197
2,30 minutes to 1h,canceled,8,122,6.557377
3,30 minutes to 1h,ended,114,122,93.442623
4,1h to 3h,canceled,54,469,11.513859
5,1h to 3h,ended,415,469,88.486141
6,3h to 12h,canceled,121,841,14.387634
7,3h to 12h,ended,720,841,85.612366
8,>12h,canceled,3047,19599,15.546712
9,>12h,ended,16552,19599,84.453288


Overall, it seems that the cancelation is not higher when there are a small amount of time between 2 reservations

Let's have a look at the previous rental id when we have the information

In [53]:
df = df.merge(
    df[['rental_id', 'delay_at_checkout_in_minutes']],
    left_on='previous_ended_rental_id',
    right_on='rental_id',
    how='left',
    suffixes=('', '_previous')
).rename(columns={'delay_at_checkout_in_minutes_previous': 'delay_previous_rental'})


df[df["previous_ended_rental_id"].notna()].head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,time_vs_previous_rental_category,rental_id_previous,delay_previous_rental
6,511639,370585,connect,ended,-15.0,563782.0,570.0,3h to 12h,563782.0,136.0
19,519491,312389,mobile,ended,58.0,545639.0,420.0,3h to 12h,545639.0,140.0
23,521156,392479,mobile,ended,NaN,537298.0,0.0,<30 minutes,537298.0,NaN
34,525044,349751,mobile,ended,NaN,510607.0,60.0,1h to 3h,510607.0,-113.0
40,528808,181625,connect,ended,-76.0,557404.0,330.0,3h to 12h,557404.0,-352.0


In [54]:
df['gap_between_checkin_chekout']=df['time_delta_with_previous_rental_in_minutes']-df['delay_previous_rental']
df['late_checking'] = ''
bins = [-np.inf, 0, np.inf]

labels = ['Late', 'Not Late']



df['late_checking'] = pd.cut(
    df['gap_between_checkin_chekout'],
    bins=bins,
    labels=labels,
    right=False  
)
df[df["previous_ended_rental_id"].notna()].head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,time_vs_previous_rental_category,rental_id_previous,delay_previous_rental,gap_between_checkin_chekout,late_checking
6,511639,370585,connect,ended,-15.0,563782.0,570.0,3h to 12h,563782.0,136.0,434.0,Not Late
19,519491,312389,mobile,ended,58.0,545639.0,420.0,3h to 12h,545639.0,140.0,280.0,Not Late
23,521156,392479,mobile,ended,NaN,537298.0,0.0,<30 minutes,537298.0,NaN,NaN,NaN
34,525044,349751,mobile,ended,NaN,510607.0,60.0,1h to 3h,510607.0,-113.0,173.0,Not Late
40,528808,181625,connect,ended,-76.0,557404.0,330.0,3h to 12h,557404.0,-352.0,682.0,Not Late


In [55]:
# Step 1: Group by and count rental_id
grouped = df.groupby(['late_checking','state'],observed=False)['rental_id'].count().reset_index()
grouped.rename(columns={'rental_id': 'count'}, inplace=True)

# Step 2: Group by the same columns and calculate the sum
sum_grouped = df.groupby(['late_checking'],observed=False)['rental_id'].count().reset_index()
sum_grouped.rename(columns={'rental_id': 'sum'}, inplace=True)

result = pd.merge(grouped, sum_grouped, on=['late_checking', 'late_checking'])

result['percentage'] = result['count']/result['sum']*100
result

,late_checking,state,count,sum,percentage
0,Late,canceled,37,218,16.972477
1,Late,ended,181,218,83.027523
2,Not Late,canceled,169,1511,11.184646
3,Not Late,ended,1342,1511,88.815354


In total, only 218 reservations out of the 21k existing were impacted by a delay. Let's check what is the gap of delay for this case

In [56]:
df_late_checkout = df[df['late_checking']=='Late']
df_late_checkout.groupby(['time_vs_previous_rental_category','state'], observed=False)['rental_id'].count().reset_index()

,time_vs_previous_rental_category,state,rental_id
0,<30 minutes,canceled,17
1,<30 minutes,ended,99
2,30 minutes to 1h,canceled,2
3,30 minutes to 1h,ended,28
4,1h to 3h,canceled,10
5,1h to 3h,ended,40
6,3h to 12h,canceled,7
7,3h to 12h,ended,13
8,>12h,canceled,1
9,>12h,ended,1


As a first approach, 30 minutes seems like the minimum gaps between 2 reservations as it covers almost half of the reservation where the car was booked in the same day. This can be linked to uncertainties in traffic leading to small delay

Mostly there were not canceled, probably because customer really needs the car and could just wait

Overall, this issue seems to be limited in numbers. To confirm that this is not a large scale issue, let's check how much are rented each cars

In [57]:
df_rental_per_car = df.groupby('car_id')['rental_id'].count().reset_index()

# Define the bins and labels
bins = [-np.inf, 1.1, 3.1, 5.1, 10.1, np.inf]
labels = ['1 rental', '2-3 rentals', '4-5 rentals', '6-10 rentals', '>10 rentals']

# Categorize the time_delta_with_previous_rental_in_minutes column
df_rental_per_car['car_category'] = pd.cut(
    df_rental_per_car['rental_id'],
    bins=bins,
    labels=labels,
    right=False  # Right bound excluded, consistent with "<30"
)

df_rental_per_car.groupby('car_category',observed=False)['rental_id'].count().reset_index()

,car_category,rental_id
0,1 rental,3659
1,2-3 rentals,2720
2,4-5 rentals,963
3,6-10 rentals,635
4,>10 rentals,166


The vast majority of cars have been rented just a few time and only a limited number of cars were rented more than 10 times. This would be interesting to have more information on the cars and on the rating as carts with high rating have more changes to be rent

In [58]:

df.groupby(['time_vs_previous_rental_category','checkin_type'], observed=False)['rental_id'].count().reset_index()

,time_vs_previous_rental_category,checkin_type,rental_id
0,<30 minutes,connect,131
1,<30 minutes,mobile,148
2,30 minutes to 1h,connect,50
3,30 minutes to 1h,mobile,72
4,1h to 3h,connect,191
5,1h to 3h,mobile,278
6,3h to 12h,connect,383
7,3h to 12h,mobile,458
8,>12h,connect,3552
9,>12h,mobile,16047


The connect seems to be overly represented in the same day reservation. In the category when there are less than 30 minutes it is almost 50/50 between connect and mobile. Probably than when there are short time between reservation, the owners of the car are more likely to be physically present for the transcation

Overall, this does not seem like a big issue as the rental by car are limited to a few rental and it rentals of the same car are very rarely close to each other

Howeer, in the context of growtwh, lets study the checkout behaviour

In [59]:
df.loc[df['delay_at_checkout_in_minutes'] < 0, 'checkout_status'] = 'Late'
df.loc[df['delay_at_checkout_in_minutes'] >= 0, 'checkout_status'] = 'Not Late'
df.loc[df['delay_at_checkout_in_minutes'].isna(), 'checkout_status'] = "Unknown"

df_group = df.groupby('checkout_status')['rental_id'].count().reset_index()
df_group['perentage']= df_group['rental_id']/df.shape[0]*100


df_group



,checkout_status,rental_id,perentage
0,Late,6820,32.003754
1,Not Late,9526,44.702018
2,Unknown,4964,23.294228


There are still 32% of the rentals that are late check-in. Lets -check the category

In [90]:
# Define the bins and labels
bins = [-np.inf, -720, -120, -60, -30, 0, 30, 60, np.inf]
labels = ['>12h late', '2h-12h late', '1-2h late', '30-60 minutes late',
          '<30 minutes late', '<30 minutes early', '<60 minutes early', '>1h early']

# Categorize the 'delay_at_checkout_in_minutes' column
df['checkout_delay_category'] = pd.cut(
    df['delay_at_checkout_in_minutes'],
    bins=bins,
    labels=labels,
    right=False  # Left-inclusive
)


# Show the first few rows
df.head()



,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,time_vs_previous_rental_category,rental_id_previous,delay_previous_rental,gap_between_checkin_chekout,late_checking,checkout_status,checkout_delay_category
0,505000,363965,mobile,canceled,NaN,NaN,721.0,>12h,NaN,NaN,NaN,NaN,Unknown,NaN
1,507750,269550,mobile,ended,-81.0,NaN,721.0,>12h,NaN,NaN,NaN,NaN,Late,1-2h late
2,508131,359049,connect,ended,70.0,NaN,721.0,>12h,NaN,NaN,NaN,NaN,Not Late,>1h early
3,508865,299063,connect,canceled,NaN,NaN,721.0,>12h,NaN,NaN,NaN,NaN,Unknown,NaN
4,511440,313932,mobile,ended,NaN,NaN,721.0,>12h,NaN,NaN,NaN,NaN,Unknown,NaN


In [91]:
df_late = df[df['checkout_status']=='Late'].groupby(['checkout_status','checkout_delay_category'],observed=False)['rental_id'].count().reset_index()
total = df_late['rental_id'].sum()
df_late['percentage'] = df_late['rental_id']/total*100
df_late

,checkout_status,checkout_delay_category,rental_id,percentage
0,Late,>12h late,244,3.577713
1,Late,2h-12h late,1454,21.319648
2,Late,1-2h late,1351,19.809384
3,Late,30-60 minutes late,1319,19.340176
4,Late,<30 minutes late,2452,35.953079
5,Late,<30 minutes early,0,0.000000
6,Late,<60 minutes early,0,0.000000
7,Late,>1h early,0,0.000000


80% of the late checkout are below a 2 hours delay

In [102]:
df[(df['checkout_status']=='Late')&(df['delay_at_checkout_in_minutes']*-1<10)&(df['checkin_type']=='mobile')].head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,time_vs_previous_rental_category,rental_id_previous,delay_previous_rental,gap_between_checkin_chekout,late_checking,checkout_status,checkout_delay_category
38,527569,314220,mobile,ended,-2.0,NaN,721.0,>12h,NaN,NaN,NaN,NaN,Late,<30 minutes late
51,531507,355554,mobile,ended,-4.0,NaN,721.0,>12h,NaN,NaN,NaN,NaN,Late,<30 minutes late
74,534827,404169,mobile,ended,-7.0,531158.0,90.0,1h to 3h,531158.0,29.0,61.0,Not Late,Late,<30 minutes late
84,535399,362659,mobile,ended,-6.0,NaN,721.0,>12h,NaN,NaN,NaN,NaN,Late,<30 minutes late
89,535738,398733,mobile,ended,-1.0,NaN,721.0,>12h,NaN,NaN,NaN,NaN,Late,<30 minutes late


In [108]:

import numpy as np
import pandas as pd
import plotly.express as px

# Thresholds
threshold_range = np.arange(0, 60 * 12, step=10)
number_mobile_rental_late = []
number_connect_rental_late = []

# Calculate numbers for each threshold
for t in threshold_range:
    number_mobile_rental_late.append(
        df[(df['checkout_status'] == 'Late') & 
           (df['delay_at_checkout_in_minutes'] * -1 < t) & 
           (df['checkin_type'] == 'mobile')].shape[0]
    )
    number_connect_rental_late.append(
        df[(df['checkout_status'] == 'Late') & 
           (df['delay_at_checkout_in_minutes'] * -1 < t) & 
           (df['checkin_type'] == 'connect')].shape[0]
    )

# Calculate 80% thresholds
mobile_total = max(number_mobile_rental_late)
connect_total = max(number_connect_rental_late)
mobile_80_threshold = threshold_range[next(i for i, count in enumerate(number_mobile_rental_late) if count >= 0.8 * mobile_total)]
connect_80_threshold = threshold_range[next(i for i, count in enumerate(number_connect_rental_late) if count >= 0.8 * connect_total)]

# Combine data into a DataFrame
plot_data = pd.DataFrame({
    'Threshold': list(threshold_range) * 2,
    'Number of Late Rentals': number_mobile_rental_late + number_connect_rental_late,
    'Type': ['Mobile'] * len(threshold_range) + ['Connect'] * len(threshold_range)
})

# Create the line plot
fig = px.line(
    data_frame=plot_data,
    x='Threshold',
    y='Number of Late Rentals',
    color='Type',
    title="Number of Late Rentals by Threshold and Type"
)

# Add vertical lines for the 80% threshold
fig.add_vline(x=mobile_80_threshold, line_dash="dash", line_color="blue", annotation_text="80% Mobile")
fig.add_vline(x=connect_80_threshold, line_dash="dash", line_color="red", annotation_text="80% Connect")

fig.show()
